<div style="text-align: center;"><br>
<img src="https://assets-global.website-files.com/62b9d45fb3f64842a96c9686/62d84db4aeb2f6552f3a2f78_Quantinuum%20Logo__horizontal%20blue.svg" width="200" height="200" /></div>

# Native Arbitrary Angle H-Series Gates

Native gates are gates on a quantum computer that the hardware physically executes. Different quantum computers may have different gates that are physically executed on the hardware. Writing a gate in a quantum circuit submitted to hardware doesn't guarantee its physical execution on the device. For instance, on H-Series quantum computers, a Hadamard gate written in the circuit is not the actual gate executed. When users submit circuits using a Hadamard gate, the gate is translated into a $U1q$ gate followed by a $Rz$ gate, which the ion trap device physically executes.

<div style="text-align: center;">
<img src="figures/native_gate_workflow.png" width="800" />
</div>

The H-Series hardware compiler handles the translation from circuits users submit to the native gates run on hardware. In the H-Series Quantum Charge-Coupled Device (QCCD) architecture, the hardware compilation includes the assignment of which physical qubit corresponds to which qubit in a circuit as well as how qubits will be transported around the device. Since transport, as well as gating, incurs a small amount of error with each operation, the H-Series compiler aims to minimize the number of gates that need to be executed. 

This article showcases: 

1. Compilation from standard 2-qubit gates and primitives to native H-Series gates.
1. prepare and submit a circuit with the $SU(4)$ gate with a [Quantum Volume Test (QVT)](https://arxiv.org/abs/1811.12926) use case. The quantum volume is an important metric to benchmark the quality of a quantum computer.

For a listing of the H-Series hardware native gates, see the following links:

* *System Model H1 Product Data Sheet* available at [System Model H1](https://www.quantinuum.com/hardware/h1)
* *System Model H2 Product Data Sheet* available at [System Model H2](https://www.quantinuum.com/hardware/h2)

**Content:**

* [Arbitrary Angle ZZ Gates](#Arbitrary-Angle-ZZ-Gates)
* [Arbitrary Angle SU(4) Gates](#Arbitrary-Angle-SU(4)-Gates)
* [Compiling to H-Series Native Gates](#compiling-to-h-series-native-gates)
* [Quantum Volume Test](#Quantum-Volume-Test)

## Arbitrary Angle ZZ Gates

Quantinuum System Model H1's native gate set includes arbitrary angle ZZ gates. This is beneficial for reducing the 2-qubit gate count for many quantum algorithms and gate sequences.

$$RZZ(\theta) = e^{-i\frac{\theta}{2}\hat{Z} \otimes \hat{Z}}= e^{-i \frac{\theta}{2}} \begin{bmatrix} 1 & 0 & 0 & 0\\ 0 & e^{-i\theta} & 0 & 0\\ 0 & 0 & e^{-i\theta} & 0\\ 0 & 0 & 0 & 1 \end{bmatrix}$$

Note that $RZZ(\frac{\pi}{2}) = ZZ()$.

In [34]:
from pytket.circuit import Circuit

circuit = Circuit(2)
circuit.ZZPhase(-0.125, 0, 1);

Quantum circuits that use the gate sequence CNOT, RZ, CNOT can be replaced with the arbitrary angle ZZ gate, shown below. Arbitrary-angle two-qubit gates can be used to improve fidelity of the output and to decrease two-qubit gate depth. Specifically, the error from arbitrary-angle two-qubit gates is less than the fixed-angle two-qubit gate for small angles. The error from both gates is the same at angle $\frac{\phi}{2}$. The error from arbitrary-angle two-qubit gates increases with angle size.

<br>

<div style="text-align: center;">
<img src="figures/rzz.png" width="500"/>
</div>

## Arbitrary Angle SU(4) Gates

The General $SU(4)$ Entangler gate is available in TKET as [`OpType.TK2`](https://tket.quantinuum.com/api-docs/circuit_class.html#pytket.circuit.Circuit.TK2). This gate is a combination of `OpType.XXPhase`, `OpType.YYPhase` and `OpType.ZZPhase`, and requires three angles as input, $\alpha$, $\beta$ and $\gamma$. The definition of the gate is provided below:

$$\begin{equation} \textrm{TK2}(\alpha, \beta, \gamma) = e^{-\frac{1}{2} i \pi \alpha (\hat{X} \bigotimes \hat{X})} \quad e^{-\frac{1}{2} i \pi \beta (\hat{Y} \bigotimes \hat{Y})} \quad e^{-\frac{1}{2} \pi \gamma (\hat{Z} \bigotimes \hat{Z})} = e^{-\frac{1}{2} i \alpha (\hat{X} \bigotimes \hat{X}) -\frac{1}{2} i \pi \beta (\hat{Y} \bigotimes \hat{Y}) -\frac{1}{2} i \pi \gamma (\hat{Z} \bigotimes \hat{Z})} \end{equation} $$

In [35]:
from pytket.circuit.display import render_circuit_jupyter
from pytket.circuit import Circuit
from sympy import Symbol

symbols = [Symbol("a"), Symbol("b"), Symbol("c")]
circuit = Circuit(2)
circuit.TK2(*symbols, *circuit.qubits)
render_circuit_jupyter(circuit)

This circuit can be converted to a QASM string using the [`circuit_to_qasm_str`](https://tket.quantinuum.com/api-docs/qasm.html#pytket.qasm.circuit_to_qasm_str) function and by specifying the Quantinuum header `hqslib1`.

In [36]:
from pytket.qasm.qasm import circuit_to_qasm_str

print(circuit_to_qasm_str(circuit, header="hqslib1"))

OPENQASM 2.0;
include "hqslib1.inc";

qreg q[2];
Rxxyyzz((a)*pi,(b)*pi,(c)*pi) q[0],q[1];



Submission of the gate requires the desired native two-qubit gate to be specified in `qnexus.QuantinuumConfig` via the keyword argument, `target_2qb_gate`. Three values can be supplied as a string:

* `TK2`: to use the arbitrary-angle $SU(4)$ gate;
* `ZZPhase`: to use the arbitrary-angle $ZZ$ gate;
* `ZZMax`: to use the fixed-angle $ZZ$ gate.

In [37]:
import qnexus

config_tk2 = qnexus.QuantinuumConfig(
    device_name="H1-1E", 
    target_2qb_gate="TK2"
)

config_zzphase = qnexus.QuantinuumConfig(
    device_name="H1-1E", 
    target_2qb_gate="ZZPhase"
)

config_zzmax = qnexus.QuantinuumConfig(
    device_name="H1-1E", 
    target_2qb_gate="ZZMax"
)

## Compiling to H-Series Native Gates

To run a circuit containing the general $SU(4)$ gate on H-Series devices, the configuration for `QuantinumConfig` needs to have `OpType.TK2` set as the native two-qubit gate. This is accomplised by setting `target_2qb_gate`, compilation and H-Series execution with `OpType.TK2` is enabled.

A new project is defined to contain all the job resources required for the arbitrary angle 2-qubit gate demonstration.

In [38]:
project = qnexus.projects.get_or_create(
    name="arbitrary-angle-gates", 
    description="Project containing job data for"
    "arbitrary-angle 2-qubit gate demonstration"
)
qnexus.context.set_active_project(project)


In [39]:
import datetime

jobname_suffix = datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")

The circuit defines one `OpType.CX` operation on 2-qubits.

In [40]:
from pytket.circuit import Circuit

circuit_cx = Circuit(2).CX(0, 1)

ref_cx = qnexus.circuits.upload(
    circuit=circuit_cx,
    name="CX-circuit",
    description="circuit containing CX gate"
)

The CX circuit is compiled using the nexus client in the code-cell below and the output circuit (with H-Series native_gates) is visualized in the subsequent code-cell. Since, the `qnexus.QuantinuumConfig` instance (specifying `OpType.ZZPhase`) is passed into the `start_compile_job` method, the output circuit will contain `OpType.ZZPhase` and other 1-qubit native H-Series gates.

In [41]:
ref_compile_job_cx = qnexus.start_compile_job(
    circuits=[ref_cx],
    name=f"cx-compile-job-{jobname_suffix}",
    optimisation_level=2,
    backend_config=config_zzphase
)

In [42]:
qnexus.jobs.wait_for(ref_compile_job_cx)
ref_compiled_circuit_cx = qnexus.jobs.results(
    ref_compile_job_cx
)[0].get_output()
render_circuit_jupyter(
    ref_compiled_circuit_cx.download_circuit()
)

The code-cell below defines a Pauli exponential over two-qubit, $e^{-i\frac{a}{2} \hat{X} \hat{Y}}$, and uploads the circuit to the nexus server.

In [43]:
from pytket.circuit import Circuit, PauliExpBox
from pytket.circuit import Pauli

from sympy import Symbol

peb = PauliExpBox([Pauli.X, Pauli.Y], -0.)
circuit_peb = Circuit(2).add_pauliexpbox(peb, circuit.qubits)

ref_peb = qnexus.circuits.upload(
    circuit=circuit_peb,
    name="PEB-circuit",
    description="circuit containing CX gate"
)

The compilation procedure with nexus will result in a compiled circuit with 1 `OpType.ZZPhase` gate and 1-qubit native H-Series gates (for basis transformation).

In [44]:
ref_compile_job_peb = qnexus.start_compile_job(
    circuits=[ref_peb],
    name=f"peb-compile-job-{jobname_suffix}",
    optimisation_level=2,
    backend_config=config_zzphase
)

In [70]:
qnexus.jobs.wait_for(ref_compile_job_peb)
ref_compiled_circuit_peb = qnexus.jobs.results(
    ref_compile_job_peb
)[0].get_output()
circuit = ref_compiled_circuit_peb.download_circuit()

## Quantum Volume Test

Quantum volume is a benchmarking test that was initially proposed by IBM ([arXiv:1811.12926](https://arxiv.org/abs/1811.12926)). It is a test that aims to verify the quality as well as the quantity of qubits on the machine. The test does this by performing rounds of single and two qubit gates between random pairs of qubits for as many rounds as qubits in the test. For example, for quantum volume of $2^N$ where $N=4$, there are 4 layers of repeated operations over 4 qubits. It verifies the quantum computer can perform quality computation with reasonable-sized circuits. The advantage to using quantum volume is that it gives users the confidence that not only do they have the number of qubits to support running their circuit, but the two-qubit gate fidelity meets the threshold to support circuits of significant depth as well. Quantum algorithms need not only qubits, but the ability to run many gates. Quantinuum has steadily been increasing the quantum volume of H-Series machines. 

In [46]:
import numpy as np

from pytket import Circuit, OpType
from pytket.circuit.display import render_circuit_jupyter

To set up the Quantum Volume test, we start by building up the repeated circuit elements. The function in the code cell below defines a TKET Circuit Box with [`pytket.circuit.CircBox`](https://tket.quantinuum.com/api-docs/circuit.html#pytket.circuit.CircBox). Circuit Boxes are useful for composing larger circuits from smaller subcircuits that utilize the same set of gates.

The Circuit box below contains a blueprint for the decomposition of a random general $SU(4)$ unitary distributed with the Haar Measure into a circuit primitive over 2 qubits. The implementation is based on [arxiv.0609050](http://arxiv.org/abs/math-ph/0609050). The random $SU(4)$ unitary is generated using [`scipy.stats.unitary_group`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.unitary_group.html#scipy-stats-unitary-group).

In [47]:
from pytket.circuit import CircBox, Circuit, Unitary2qBox
import numpy as np
from scipy.stats import unitary_group


def haar_random_su4_box() -> CircBox:
    r = unitary_group.rvs(dim=4)
    r[0, :] /= np.linalg.det(r)
    circuit = Circuit(2)
    box = Unitary2qBox(r)
    circuit.add_unitary2qbox(box, 0, 1)
    return CircBox(circuit)

Circuit Boxes can be visualized utilizing the `get_circuit` function.

In [48]:
from pytket.circuit.display import render_circuit_jupyter

circuit = haar_random_su4_box().get_circuit()
render_circuit_jupyter(circuit)

#### `OpType.TK2`

In the cell below, the CircBox is inspected and further optimisations are applied. `DecomposeBoxes` simplifies `OpType.CircBox` operations into the underlying gate operations recursively. Further decompositions  are performed with the `KAKDecomposition` function. We specify use the `OpType.TK2` gate in this function call as the target 2-qubit gate after decomposition. 

The `pytket` sequence pass, consists of two passes:
* [`pytket.passes.DecomposeBoxes`](https://tket.quantinuum.com/api-docs/passes.html#pytket.passes.DecomposeBoxes)
* [`pytket.passes.KAKDecomposition`](https://tket.quantinuum.com/api-docs/passes.html#pytket.passes.KAKDecomposition)

Both passes are arguments to [`pytket.passes.SequencePass`](https://tket.quantinuum.com/api-docs/passes.html#pytket.passes.SequencePass). `SequencePass` allows both passes to be applied to the input circuit with one `apply` call.

In [49]:
from pytket.passes import DecomposeBoxes, KAKDecomposition, SequencePass
from pytket.circuit import OpType

sequence_pass = SequencePass(
    [DecomposeBoxes(), KAKDecomposition(target_2qb_gate=OpType.TK2)]
)

In [50]:
sequence_pass.apply(circuit)
render_circuit_jupyter(circuit)

Now we're ready to set up a Quantum Volume test circuit using the Circuit Box we created. The steps to set up the circuit are straightforward using the `CircBox` instances to build up the full circuit.

In [51]:
circuit = Circuit(4)
for _ in range(4):
    permutation = np.random.permutation(circuit.qubits)
    for i in range(0, 4, 2):
        box = haar_random_su4_box()
        circuit.add_circbox(box, [permutation[i], permutation[i + 1]])
circuit.measure_all();

The `SequencePass` defined in the code cell in the previous subsection is used below on the QVT circuit and we can see the full set of gates in the circuit as well as the result of optimizations performed by `KAKDecomposition`.

In [52]:
from pytket.circuit.display import render_circuit_jupyter

sequence_pass.apply(circuit)
render_circuit_jupyter(circuit)

In [53]:
ref_su4 = qnexus.circuits.upload(circuit=circuit, name="su4_circuit")

In [55]:
ref_compile_job_tk2 = qnexus.start_compile_job(
    circuits=[ref_su4],
    name=f"compilation-tk2-job-{jobname_suffix}",
    backend_config=config_tk2,
    optimisation_level=2
)

In [56]:
qnexus.jobs.wait_for(ref_compile_job_tk2)
ref_compile_circuit_tk2 = qnexus.jobs.results(ref_compile_job_tk2)[0].get_output()

In [57]:
compiled_circuit_tk2 = ref_compile_circuit_tk2.download_circuit()

In [58]:
from pytket.circuit.display import render_circuit_jupyter

render_circuit_jupyter(compiled_circuit_tk2)

In [61]:
ref_execute_job_tk2 = qnexus.start_execute_job(
    circuits=[ref_compile_circuit_tk2],
    name=f"execution-tk2-job-{jobname_suffix}",
    backend_config=config_tk2,
    n_shots=[100]
)

In [60]:
qnexus.jobs.wait_for(ref_execute_job_tk2)
result_tk2 = qnexus.jobs.results(ref_execute_job_tk2)[0].download_result()

#### `OpType.ZZPhase`

In [62]:
ref_compile_job_zzphase = qnexus.start_compile_job(
    circuits=[ref_su4],
    name=f"compilation-zzphase-job-{jobname_suffix}",
    backend_config=config_zzphase,
    optimisation_level=2
)

In [63]:
qnexus.jobs.wait_for(ref_compile_job_zzphase)
ref_compile_circuit_zzphase = qnexus.jobs.results(ref_compile_job_zzphase)[0].get_output()

ResourceFetchFailed: Failed to fetch resource with status code: 500, message: {'message': 'Internal Server Error - Unknown'}

In [ ]:
compiled_circuit_zzphase = ref_compile_circuit_zzphase.download_circuit()

In [ ]:
from pytket.circuit.display import render_circuit_jupyter

render_circuit_jupyter(compiled_circuit_zzphase)

In [ ]:
ref_execute_job_zzphase = qnexus.start_execute_job(
    circuits=[ref_compile_circuit_tk2],
    name=f"execution-zzphase-job-{jobname_suffix}",
    backend_config=config_zzphase,
    n_shots=[100]
)

In [ ]:
qnexus.jobs.wait_for(ref_execute_job_zzphase)
result_zzphase = qnexus.jobs.results(ref_execute_job_zzphase)[0].download_result()

<div align="center"> &copy; 2024 by Quantinuum. All Rights Reserved. </div>